In [76]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [77]:
dossier="/Copy/Cours/Bordeaux/L1-UE1/Kalaba-15/02-PseudoLatin"
enonce="Theme"
theme="Traductions"

In [78]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [79]:
from MoodleXML import *
import yaml

In [80]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=False
titreNoEX=False

In [81]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with open(csvFichier, 'r') as file:
    donneesLignes=file.readlines()

In [82]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
else:
#    print "sans #EX;"
    donneesNoEX()

In [83]:
categorie=configuration[u"Catégorie"]
#donnees=configuration[u"Données"]
consigne=configuration[u"Boucle"][u"contenu"]
boucleWrap=[configuration[u"Boucle"][u"début"],configuration[u"Boucle"]["fin"]]
boucleCLOZE=configuration[u"Boucle"][u"Cloze"]
globalWrap=[configuration[u"Global"][u"début"],configuration[u"Global"]["fin"]]
if "Conclusion" in configuration:
    conclusion=[configuration[u"Conclusion"][u"début"],configuration[u"Conclusion"]["contenu"],configuration[u"Conclusion"]["fin"]]
    conclusionCLOZE=configuration[u"Conclusion"][u"Cloze"]
else:
    conclusion=["","",""]
    conclusionCLOZE=[]
questions=XMLClozes(categorie)
questionsConsigne=ClozeConsigne(consigne,boucleWrap=boucleWrap,globalWrap=globalWrap,conclusion=conclusion)
serie=ClozeSerie(configuration[u"Boucle"][u"Cloze"],conclusionCLOZE)

In [84]:
for exercices in donnees:
#    print "exercices",exercices
    for exercice in exercices:
#        print "exercice",exercice
        exerciceBoucle=exercices[exercice]["boucle"]
        exerciceConclusion=exercices[exercice]["conclusion"]
        serie.addExercice(Exercice(exerciceBoucle,exerciceConclusion,exercice.decode('utf8')))

In [85]:
for exercice in serie.makeSerie(questionsConsigne):
#    print exercice.corps
    questions.addExercice(exercice)

In [86]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getClozes())

In [87]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [88]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())